In [ ]:
import osmnx as ox
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import json

def column_usefulness(df,perc=50):
    print(len(df))
    for i,n in df.isna().sum().sort_values().iteritems():
        if n<perc/100*len(df):
            print(f"{i}: {(len(df)-n)/len(df):.2%}")

# Create CityScope Table

In [ ]:
from brix import Grid_maker, Handler
table_name = 'ntt_rome'
lat,lon = 41.835720, 12.498408 #NTTD IT Rome Coordinates (top-left point)
G = Grid_maker(table_name, lat, lon, 
               cell_size=75, nrows=20, ncols=20, rotation=0, 
               crs_epsg=None, quietly=False)

In [ ]:
grid_geo=G.get_grid_geojson()
grid_gdf=gpd.read_file(json.dumps(grid_geo))

## Assign Land Uses to grid cells

Have to ignore non-polygon types

In [ ]:
db_valid= gpd.read_file("zones/ntt.json")   #as is scenario (from OSM)

In [ ]:
grid_over_lu=gpd.overlay(grid_gdf, db_valid)

In [ ]:
grid_gdf=grid_gdf.merge(grid_over_lu[['id', 'section']], 
                        left_on='id', right_on='id', how='left')

In [ ]:
grid_gdf['section'] = grid_gdf['section'].fillna(value='none')

In [ ]:
grid_gdf['name']=grid_gdf['section'].copy()
grid_gdf=grid_gdf.drop(columns=['section'])

## Convert back to dict and add metadata

In [ ]:
grid_gdf

In [ ]:
new_grid_geo=grid_gdf.__geo_interface__

In [ ]:
new_grid_geo

In [ ]:
new_grid_geo['properties']=grid_geo['properties']

## Edit the 'Types' list

In [ ]:
types={
    "educational":
    {
        "color":
        [
            1,#
            152,
            189,
            255 #180
        ],
        "height":
        [
            0,
            4,
            40
        ]
    },
    "institutional":
    {
        "color":
        [
            232,#
            254,
            181,
            255 #180
        ],
        "height":
        [
            0,
            4,
            40
        ]
    },
    "industrial":
    {
        "color":
        [
            213,#
            2,
            85,
            255 #180
        ],
        "height":
        [
            0,
            4,
            40
        ]
    },
    
    "office":
    {
        "color":
        [
            227,
            26,
            28,
            255 #180
        ],
        "height":
        [
            0,
            4,
            40
        ]
    },
    "park":
    {

        "color":#
        [
            0,
            168,
            157,
            180 #180
        ],
        "height":
        [
            0,
            4,
            40
        ],
    },
    "residential":
    {
        "color":
        [
            254,#
            173,
            84,
            180 #180
        ],
        "height":
        [
            0,
            4,
            40
        ]
    },
    "retail":
    {
        "color":
        [
            251,
            154,
            153,
            180 #180
        ],
        "height":
        [
            0, # min
            4, # default
            40 # max
        ]
    }, 
    "none":
    {
        "color":
        [
            0,
            0,
            0,
            0
        ],
        "height": 0
    }
}

In [ ]:
new_grid_geo['properties']['types']=types

In [ ]:
G.geojson_object=new_grid_geo

In [ ]:
try:
    G.commit_grid()
except:
    pass

In [ ]:
#Table is now accessible at https://cityscope.media.mit.edu/CS_cityscopeJS/?cityscope=ntt_rome

After changing the grid on the front-end you can reset it using the following:

In [ ]:
H = Handler('ntt_rome')
H.reset_geogrid_data()